# Ejercicio 1 - Mejoras en el Desempeño de Modelos de Clasificación

Este ejercicio se centra en el entrenamiento y evaluación de algunos modelos modelos de clasificación, sobre un conjunto de datos de campañas de marketing de productos bancarios, aplicando algunas estrategias de mejora del rendimiento de los modelos. Al considerar técnicas previas a los modelos se busca demostrar que el desempeño de éstos depende fuertemente del pre-procesamiento de los datos, más que un afinamiento de sus parámetros.

## Contexto: Análisis de éxito en campaña de marketing

Fuente: https://archive.ics.uci.edu/ml/datasets/Bank+Marketing

El *dataset* para este ejercicio contiene resultados de personas contactadas por una campaña de marketing y que compraron la oferta (atributo "OK" = yes o no), con cerca de 41.200 registros de personas.

Este conjunto de datos (abierto para este tipo de usos instruccionales), consiste en 20 atributos y 1 clase de etiquetas (totalizando 21 columnas) y corresponde a los datos de una campaña telefónica en Portugal, ofreciéndoles la contratación de un depósito a plazo. En varios casos, un cliente fue contactado varias veces antes de aceptar (OK = yes).

Algunos de los atributos relevantes son (combinando atributos categóricos, con numéricos):
* **Datos personales**: Edad, Ocupación, Estado Civil, Nivel de Educación.
* **Datos financieros**: Su casa tiene crédito hipotecario, default: si el crédito ha caído en quiebra; tiene un crédito de consumo.
* **Datos de contactos de la campaña actual**: Tipo de Comunicación (celular o teléfono fijo); Mes del último contacto; Día de la semana del contacto; duración de la llamada (segundos); Contacto: N° de contactos durante la campaña; DíasAtrás: días transcurridos desde último contacto; Resultado: resultado de la última llamada (falló, no-existe, éxito)
* **Datos socioeconómicos**: EmpTasaVar: tasa de variación de empleabilidad; IPC: índice de precios consumidor mensual; ICC: índice de confianza consumidor mensual; Euribor3m: tasa euribor de 3 meses indicador diario; NumEmpleados: cantidad de gente empleada, en indicador trimestral.

Esta adaptación en particular, por Rodrigo Sandoval y el equipo de R:Solver (RSolver.com), enfrenta diferentes objetivos de aprendizaje dentro de los cursos de Big Data y Machine Learning.



## Instrucciones Generales
En este caso, se busca entender el comportamiento y desempeño de diferentes modelos de clasificación sobre este conjunto de datos, para predecir la variable de interés: **OK**, que servirá para predecir en casos futuros, según los datos de contactabilidad de un cliente, si el cliente aceptará o no contratar el depósito a plazo.

Todos los alumnos deben contestar las preguntas que se indican en las secciones de "Preguntas", más adelante, las que se enfocan en tareas de pre-procesamiento del conjunto de datos, para lograr mejoras en el desempeño del modelo de predicción categórica. Se puede recurrir a ejercicios de otras fuentes, así como al material de clases. Deben entregar un informe simple (de 1 ó 2 páginas) en formato PDF solamente, en el que se incluyen los nombres de los integrantes ordenados alfabéticamente por apellido dentro de la primera página.

La plantilla de informe está en http://dcc.rsolver.com/dcc/docs/InformeActividad.docx

El informe en formato PDF debe ser subido por sólo uno de los integrantes a la siguiente URL

http://aiker.rsolver.com/aiker/DocUpload.aspx (*)

(*) Sólo si hay problemas en la carga, enviar el PDF a rsandova@ing.puc.cl y cc: ayudante@aiker.ai, pero no se requiere como respaldo de la entrega.

## Paso 1: Instalación de las librerías de modelos de clasificación

Esto se ejecuta sólo una vez al comienzo de la sesión de cada persona. No se necesita volver a ejecutar con cada nueva prueba del resto de los scripts o bloques.

In [ ]:
install.packages('e1071')
install.packages('caret')
install.packages('rpart')
install.packages('rpart.plot')
install.packages('randomForest')


## Paso 2: Carga de los datos

En este bloque se cargan los datos desde la URL de origen y luego muestra un encabezado con las primeras filas del dataset, para demostrar la disponibilidad de los datos.

Esto también puede ser ejecutado una sola vez, o volver a ejecutar cuando nuevamente se necesite trabajar con los datos originales.

In [ ]:
# Se declara la URL de dónde obtener los datos
theUrlMain <- "http://www.rsolver.com/dcc/docs/bank-additional-full.csv"

# Se declaran los nombres de las columnas
columnas = c("Edad","Ocupación","EstadoCivil","Educación","Default","Hipotecario","Consumo","Contacto","Mes","Día",
             "Duración","NumContactos","DíasAtrás","Previo","ResultadoPrevio", "EmpTasaVar", "IPC", "ICC", "Euribor3m", "NumEmpleados", "OK")

# Se cargan datos principales a una estructura (marketing.data), asignando nombres de atributos a las columnas.
# Nótese que se incluye la conversión de valores "unknown" a "NA" para facilitar la gestión vacíos más adelante.
marketing.data <- read.table(file = theUrlMain, header = TRUE, sep = ";", col.names = columnas, na.strings=c("unknown","NA"))

# Se muestran las primeras líneas del dataset, incluyendo nombres asignados a las columnas.
# Nótese que ya no hay "unknown", sino que sólo "NA", lo que permite a las operaciones is.na() y na.omit() operar fácilmente
head(marketing.data, 20)
dim(marketing.data)


## Paso 3: Complemento de Clasificación Inicial

A continuación se implementa un proceso de clasificación inicial, sin alterar el conjunto de ejemplos y teniendo una proporción genérica de hold-out, con 60% de ejemplos para entrenamiento y 40% de ejemplos para evaluación. Se termina ejecutando un Decision Tree, mostrando sus indicadores de desempeño.

Esta ejecución de un modelo inicial sirve de comparación, evaluando cuánto influyen las mejoras al dataset y entrenamiento, que son parte de los ejercicios, en el desempeño de otros modelos más adelante.

**Análisis del desempeño del modelo de referencia**

Nótese que se muestra el desempeño del modelo con los datos de entrenamiento y a continuación con los datos de evaluación. Dentro de lo esperable, el accuracy para el conjunto entrenamiento tiende a ser levemente mejor.

Adicionalmente, se puede ver que el sensitivity (desempeño de la clase positiva o "yes") es notoriamente menor que el specificity, lo que evidencia un sobreajuste a responder "no" de parte del modelo, que se interpreta como una limitación o desempeño deficiente, que se espera poder mejorar.

Finalmente, el gráfico del Árbol de Decisión muestra que, de todos los atributos del dataset, los dos más significativos para armar las reglas de decisión de este Árbol de Decisión, son "NumEmpleados", "Duración" y "DíasAtrás".

In [ ]:
# Primero se separa el conjunto de ejemplos (marketing.data) en 2,
# uno de entrenamiento y otro de evaluación usando la técnica hold-out,
# que consiste en crear dos conjuntos disjuntos en una proporción dada

ratio = sample(1:nrow(marketing.data), size = 0.60*nrow(marketing.data)) # Proporción: 60% / 40%
training.data = marketing.data[ratio,]
testing.data = marketing.data[-ratio,]

# Se muestra la cantidad de ejemplos de cada uno de los dos conjuntos (en la proporción indicada)
dim(training.data)
dim(testing.data)

# Luego se entrena un modelo de clasificación Decision Tree usando el conjunto Train
# y posteriormente se evalúa con predict() para obtener los indicadores de desempeño

library(caret)
library(rpart)
library(rpart.plot)

# Decision Tree
DT_model <- rpart(as.factor(OK) ~ ., data=training.data, method="class", minbucket=10)

cat("\n****Desempeño Decision Tree en conjunto de entrenamiento****\n")
DT_predict_train <- predict(DT_model, training.data, type = "class")
confusionMatrix(DT_predict_train, as.factor(training.data$OK), positive="yes")

cat("\n---------------------------------------------------------\n")
cat("\n****Desempeño Decision Tree en conjunto de evaluación****\n")
DT_predict_test <- predict(DT_model, testing.data, type = "class")
confusionMatrix(DT_predict_test, as.factor(testing.data$OK), positive="yes")
rpart.plot(DT_model)




---


## Ejercicio: Limpieza, balanceo de datos y preparación conjuntos de entrenamiento y evaluación

El siguiente código prepara los datos para el entrenamiento y evaluación, realizando algunas operaciones de limpieza de datos y balanceo entre clases. Hay diferentes elementos de análisis y acciones de mejora del dataset, entre los que se incluye (pero no reduce a):

*   Ya incluido: eliminación de los registros incompletos (que tienen un 'NA' en alguna columna) con na.omit(). Pero adicionalmente se puede optar por lo siguiente.
*   (A) **Eliminar algunas columnas**, considerando aquellas que tengan datos de bajo aporte o interés, o cuya distribución de valores sea irregular, lo que llevaría a un modelo a dar predicciones ruidosas. (Como referencia de análisis de posibles columnas candidatas a ser eliminadas, observe el caso de DíasAtrás = 999, que se define como el "valor nulo", que implica que no se ha hecho contacto anteriormente).
*   (B) **Balance entre clases**, considerando que el dataset original tiene una gran proporción de clientes que NO compraron, a diferencia de los que SI, se incluye una variable balance_ratio, donde 1.0 implica igualar la cantidad de ejemplos de NO y SI, mientras que agrandar ese valor implica agrandar la cantidad de ejemplos NO por sobre los YES. Nótese que este balance es sólo relevante para el conjunto de entrenamiento y no para el de evaluación o test.

**Pregunta 1**  (1 punto): En (A) ¿cuáles columnas se eliminaron logrando mejorar el desempeño de los modelos? (Considere al menos 2)

**Pregunta 2**  (2 puntos): En (B) ¿qué proporción (Probar: 0.8, 0.9, 1.0, 1.2, 1.5) de ejemplos NO/YES se eligió y por qué? Entonces, ¿de qué dimensiones quedó el dataset de entrenamiento modificado?

Al contestar incluya una tabla de ejecuciones comparadas, para demostrar que su decisión está basada en resultados de evidencia.



In [ ]:
# Primero, se eliminan los registros que tienen algún NA (antes: 'unknown')
marketing.clean <- na.omit(marketing.data)
dim(marketing.clean) # Sólo quedan poco más de 30.000 filas (de las 41.000 originales)


# (A) ¿Cuáles columnas se pueden eliminar para mejorar la calidad de los datos? (Consider al menos 2)
# Se elimina columnas con la siguiente instrucción de ejemplo:
# marketing.clean$ICC <- NULL


dim(marketing.clean)


In [ ]:
# Aquí se arman dos subconjuntos con los datos de cada una de las dos clases.
# Se pueden ver los respectivos tamaños al terminar, evidenciando un desbalance.
clean.data.YES <- marketing.clean[marketing.clean$OK == 'yes',]
clean.data.NO <- marketing.clean[marketing.clean$OK == 'no',]
dim(clean.data.YES) # Se ve que este es el conjunto más pequeño
dim(clean.data.NO)

In [ ]:
# (B) Se balancean las clases para entrenar: se busca acercar la cantidad de ejemplos positivos, con los negativos.
# Para esto se puede definir la cantidad de ejemplos de la clase más abundante (NO)
# en una proporción (balance_ratio) de la cantidad de registros de la clase menos abundante (YES)
# balance_ratio = 1.0 implica la misma cantidad para NO y para YES ¿Cuál es la mejor? (Probar: 0.8, 0.9, 1.0, 1.2, 1.5)
balance_ratio <- 1.0

clean.subdata.YES <- clean.data.YES  # No se aplica sample(): se usan todos los ejemplos de la clase OK (que es la que tiene menos ejemplos)
clean.subdata.NO <- clean.data.NO[sample(nrow(clean.data.NO), balance_ratio*dim(clean.data.YES)[1]), ]

# Muestra cantidad de ejemplos contenidos en cada subconjunto
dim(clean.subdata.YES)
dim(clean.subdata.NO)

# Se juntan para el conjunto de referencia, ahora más balanceado
clean.subdata <- rbind(clean.subdata.YES, clean.subdata.NO)

# Ahora se configuran los conjuntos de entrenamiento y testing en una proporción
# (por ej: 0.85 = 85% para training y el resto para evaluación o testing)
ratio = sample(1:nrow(clean.subdata), size = 0.85*nrow(clean.subdata))
training.data = clean.subdata[ratio,]
testing.data = clean.subdata[-ratio,]

# Se comparan los tamaños de ejemplos para entrenamiento y evaluación.
dim(training.data)
dim(testing.data)

## Complemento: Implementación de un modelo Random Forest para comparar

Esta sección implementa un Random Forest, como modelo de clasificación de referencia, cuyo desempeño será comparado con el Árbol de Decisión anterior, que no tuvo aplicación de las mejoras al dataset del ejercicio anterior.

**Pregunta 3** (1,5 puntos): Si se compara con el desempeño del Árbol de Decisión en el paso 3 anterior ¿Qué diferencia fundamental (ventaja/desventaja) se ve en los indicadores de desempeño y por qué se podría haber logrado esta diferencia?

**Pregunta 4** (1,5 puntos): Al comparar el desempeño del modelo con los datos de entrenamiento y con los datos de evaluación, se puede ver una diferencia importante. ¿Cómo se interpreta esta diferencia y, en teoría, cómo se podría resolver?



**Random Forest**

In [ ]:
library(randomForest)
library(caret)

# Random Forest
RF_model <- randomForest(as.factor(OK) ~ ., data=training.data, method="class")

cat("\n****Desempeño Random Forest en conjunto de entrenamiento****\n")
RF_predict_train <- predict(RF_model, training.data, type = "class")
confusionMatrix(RF_predict_train, as.factor(training.data$OK), positive="yes")

cat("\n---------------------------------------------------------\n")
cat("\n****Desempeño Random Forest en conjunto de evaluación****\n")
RF_predict <- predict(RF_model, testing.data, type = "class")
confusionMatrix(RF_predict, as.factor(testing.data$OK), positive="yes")


---

**Complemento: Ejercicio de comprobación manual**


Para verificar que alguno de los modelos realmente predice correctamente, se comprueba con los datos de una persona en particular, pidiendo la predicción al modelo. A continuación hay dos ejemplos, que se pueden modificar para ver su resultado, cambiando valores y también, cambiando el modelo a utilizar en la predicción. No se necesita modificar, ni comentar esta parte en la entrega.

In [ ]:
# Ejemplo 1: La predicción debería ser "YES"
sample_x <- clean.subdata[1,]
sample_x[1,1] <- 32       # Edad
sample_x[1,2] <- 'admin.' # Ocupación
sample_x[1,3] <- 'single' # EstadoCivil
sample_x[1,4] <- 'university.degree'  # Educación
sample_x

prediction <- predict(RF_model, sample_x)
prediction


# Ejemplo 2: La predicción debería ser "NO"
sample_x2 <- clean.subdata[1,]
sample_x2[1,1] <- 42
sample_x2[1,11] <- 10   # Duración
sample_x2[1,19] <- 5000  # NumEmpleados
sample_x2

prediction <- predict(RF_model, sample_x2)
prediction
